# <span style="color:darkblue"> Basic NLP toolchain</span> 

<ul> <li><a href="#-Basic-NLP-toolchain"> Basic NLP toolchain</a></li>
  <ul><li><a href="#A-Short-Introduction-and-Tutorial">A Short Introduction and Tutorial</a></li>
  <li><a href="#-Text-segmentation-"> Text segmentation </a></li>
    <ul><ul><li><a href="#Tokens-vs-words">Tokens vs words</a></li>
    <li><a href="#Sentences">Sentences</a></li>
  </ul></ul><li><a href="#Morphological-analysis">Morphological analysis</a></li>
    <ul><ul><li><a href="#Accessing-details-of-the-morphological-analysis">Accessing details of the morphological analysis</a></li>
    <li><a href="#Parameters-of-morphological-analysis:-disambiguation,-guessing...">Parameters of morphological analysis: disambiguation, guessing...</a></li>
     <ul><li><a href="#Note:">Note:</a></li>
     <li><a href="#Warning:">Warning:</a></li>
    </ul><li><a href="#Unknown-words">Unknown words</a></li>
     <ul><li><a href="#Remarks-on-morphological-analysis:">Remarks on morphological analysis:</a></li>
  </ul></ul></ul><li><a href="#-Examples"> Examples</a></li>
    <ul><ul><li><a href="#Example-1:-Finding-all-different-nouns-from-the-text">Example 1: Finding all different nouns from the text</a></li>
    <li><a href="#Example-2:-Finding-all-sentences-that-contain-an-infinitive-verb">Example 2: Finding all sentences that contain an infinitive verb</a></li>
  </ul></ul><li><a href="#-Further-details-"> Further details </a></li>
</ul></ul>

Online documentation is best viewed with https://nbviewer.jupyter.org/

## <span style="color:blue">A Short Introduction and Tutorial</span> 

This tutorial gives an overview of how to use EstNLTK for the basic analysis of text: splitting it into linguistically meaningful units - words, sentences, - and performing morphological analysis. These steps are necessary in tackling most language-related problems: if we are able to extract words and sentences from text and filter them by lemmas, part-of-speech tags and morphological forms, we can solve numerous tasks, e.g. automatically find example sentences of different grammatical constructions from large corpora, compose word/lemma frequency lists, compare texts in terms of sentence lengths/structures, etc.

The most important class in Estnltk is Text, which is essentally the main interface for doing everything Estnltk is capable of. To use it, we have to import it:

In [1]:
from estnltk import Text

To start working on our text, we have to create a new Text class object of it. Let's use a simple sentence  as an example:

In [2]:
text = Text("Müüja tatsas rahulikult külmiku juurde.")

In [3]:
text

text
Müüja tatsas rahulikult külmiku juurde.


The basic way to use EstNLTK toolchain is to use the tag_layer() method that automatically segments the text and performs morphological analysis. From its output, we can see which layers have been tagged on text, which attributes the layers have and how many elements belong to every layer (column span count). The details about each layer come in Section B.

In [4]:
text.tag_layer()

Text(text='Müüja tatsas rahulikult külmiku juurde.')

##  <span style="color:purple"> Text segmentation </span>

One of the most basic tasks of any NLP pipeline is text segmentation: splitting the text into smaller meaningful units - words, sentences, paragraphs, etc. This might seem like a trivial task at first - aren't words separated by spaces and sentences by full stops? And yes, question marks and exclamation marks. However, if we take an existing text, we will see that there are lots of exceptions to these rules. Therefore, EstNLTK has dedicated methods for these kinds of tasks that try to tackle the frequent segmentation issues.

###  Tokens vs words
To make a distinction between properly tagged words (incl. punctuation, abbreviations, e-mail addresses, etc) and elements in text that are separated from each other by whitespace (or not... in case of punctuation), we use the term 'tokens' for the latter. For the most part, tokens overlap with words, but a token might also be a part of a word: in later analysis, tokens are not broken into any smaller parts, but only joined if necessary. If we look at our first example above, we can see that the number of words and tokens is equal. However, there are cases where some tokens are joined into one word:

In [5]:
text = Text('Mis aias sa-das 3me sorti s-saia?')
text.tag_layer()

Text(text='Mis aias sa-das 3me sorti s-saia?')

As we can see, in this (quite weird) example sentence, there are 11 tokens but 7 words. To see the tokens (or words for that matter) that have been tagged on text using the tag_layer() method, we can either use the Text object as a typical Python dict:

In [6]:
text['tokens']

Layer(name='tokens', attributes=(), spans=SL[Span('Mis', [{}]),
Span('aias', [{}]),
Span('sa', [{}]),
Span('-', [{}]),
Span('das', [{}]),
Span('3me', [{}]),
Span('sorti', [{}]),
Span('s', [{}]),
Span('-', [{}]),
Span('saia', [{}]),
Span('?', [{}])])

Or, we can use the class variable 'tokens' 

In [7]:
text.tokens

Layer(name='tokens', attributes=(), spans=SL[Span('Mis', [{}]),
Span('aias', [{}]),
Span('sa', [{}]),
Span('-', [{}]),
Span('das', [{}]),
Span('3me', [{}]),
Span('sorti', [{}]),
Span('s', [{}]),
Span('-', [{}]),
Span('saia', [{}]),
Span('?', [{}])])

To get words - smallest meaningful units of language - some tokens might need to be combined. That's why there are layers `tokens` and `compound_tokens` which are combined together to create words. This happens when the raw text needs some kind of normalization in order to comply with standard ortography. In addition to the hyphenated words as in the previous example, also some numbers ( _'10 000'_ ), e-mail addresses ( 'example@example.com' ), abbreviations ( _'s.t.'_ ) and other entities that have been tagged as separate tokens have to be joined together.

We can see and use `words` layer (and all the other layers) the same way as the `tokens` layer:

In [8]:
text.words

Layer(name='words', attributes=('normalized_form',), spans=SL[Span('Mis', [{'normalized_form': None}]),
Span('aias', [{'normalized_form': None}]),
Span('sa-das', [{'normalized_form': 'sadas'}]),
Span('3me', [{'normalized_form': None}]),
Span('sorti', [{'normalized_form': None}]),
Span('s-saia', [{'normalized_form': 'saia'}]),
Span('?', [{'normalized_form': None}])])

### Sentences

A sentence is a list of sequential words and so the sentence layer is a list of lists of words. This means that first, the text is split into words, and then, sentence borders are determined so that no sentence border would end up inside a word. 

Let's see an example that has multiple sentences:

In [9]:
text = Text('''Ka köögis oli kõik endine: vana elektripliit, koorunud värviga ahjutruup ja vanamehe töövorm — rippumas ikka sealsamas ukse küljes nagis. Jälk. Köögi akna all laual oli vanaaegne arvuti. Juba aastaid. Selline kaasaskantav väike kastike, mille klaviatuur ekraani ette kinnitus. See oli sini-valge pildi ja DOS-opsüsteemiga mänguasi.''')

In [10]:
text.tag_layer()

Text(text='Ka köögis oli kõik endine: vana elektripliit, koorunud värviga ahjutruup ja vanamehe töövorm — rippumas ikka sealsamas ukse küljes nagis. Jälk. Köögi akna all laual oli vanaaegne arvuti. Juba aastaid. Selline kaasaskantav väike kastike, mille klaviatuur ekraani ette kinnitus. See oli sini-valge pildi ja DOS-opsüsteemiga mänguasi.')

We can see the text split into sentences by using the `sentences` class variable:

In [11]:
text.sentences

Layer(name='sentences', attributes=(), spans=SL[EnvelopingSpan(['Ka', 'köögis', 'oli', 'kõik', 'endine', ':', 'vana', 'elektripliit', ',', 'koorunud', 'värviga', 'ahjutruup', 'ja', 'vanamehe', 'töövorm', '—', 'rippumas', 'ikka', 'sealsamas', 'ukse', 'küljes', 'nagis', '.'], [{}]),
EnvelopingSpan(['Jälk', '.'], [{}]),
EnvelopingSpan(['Köögi', 'akna', 'all', 'laual', 'oli', 'vanaaegne', 'arvuti', '.'], [{}]),
EnvelopingSpan(['Juba', 'aastaid', '.'], [{}]),
EnvelopingSpan(['Selline', 'kaasaskantav', 'väike', 'kastike', ',', 'mille', 'klaviatuur', 'ekraani', 'ette', 'kinnitus', '.'], [{}]),
EnvelopingSpan(['See', 'oli', 'sini-valge', 'pildi', 'ja', 'DOS-opsüsteemiga', 'mänguasi', '.'], [{}])])

## <span style="color:purple">Morphological analysis</span>

In linguistics, morphology is the identification, analysis, and description of the structure of a given language’s morphemes and related linguistic units, such as root words, lemmas, suffixes, parts of speech etc. When we are processing a morphologically rich language - that Estonian certainly is -, getting this kind of information is essential for even the simplest tasks. For example, if we want to find all the mentions of 'maja' from the corpus, we are probably not eager to spell out the 27 different forms that we are interested in ( _'maja'_ , _'majale'_ , _'majadega'_ ...), but we also do not want to get things like _'majandus'_ or _'majakas'_ . If we have morphologically analysed the text, we can just state that we are interested in the lemma _'maja'_ .

Estnltk wraps [Vabamorf's](https://github.com/Filosoft/vabamorf) morphological analyzer. Morphological analysis is also performed with no extra hassle when we use the tag_layer() method on our text. When we look at the text object, we can see that there is the morph_analysis layer and it has several attributes: lemma, root, etc.

In [12]:
text = Text("Aga kõik juhtus iseenesest.").tag_layer()

In [13]:
text

Text(text='Aga kõik juhtus iseenesest.')

Therefore, we can either view the whole analysis as a table:

In [14]:
text['morph_analysis']

Layer(name='morph_analysis', attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech'), spans=SL[Span('Aga', [{'normalized_text': 'Aga', 'lemma': 'aga', 'root': 'aga', 'root_tokens': ['aga'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'J'}]),
Span('kõik', [{'normalized_text': 'kõik', 'lemma': 'kõik', 'root': 'kõik', 'root_tokens': ['kõik'], 'ending': '0', 'clitic': '', 'form': 'pl n', 'partofspeech': 'P'}, {'normalized_text': 'kõik', 'lemma': 'kõik', 'root': 'kõik', 'root_tokens': ['kõik'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'P'}]),
Span('juhtus', [{'normalized_text': 'juhtus', 'lemma': 'juhtuma', 'root': 'juhtu', 'root_tokens': ['juhtu'], 'ending': 's', 'clitic': '', 'form': 's', 'partofspeech': 'V'}]),
Span('iseenesest', [{'normalized_text': 'iseenesest', 'lemma': 'iseenesest', 'root': 'ise_enesest', 'root_tokens': ['ise', 'enesest'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'D'}]),
Span('.', [{'normalized_text': '.', 'lemma': '.', 'root': '.', 'root_tokens': ['.'], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}])])

For explanations of categories used in `partofspeech` and `form` attributes, please see: [tables_of_morphological_categories.ipynb](../nlp_pipeline/B_morphology/00_tables_of_morphological_categories.ipynb).

You can also convert morphological analysis category names to Estonian:

In [15]:
# Add a morph layer that has Estonian category names
text.tag_layer(['morph_analysis_est'])

# Browse the layer
text['morph_analysis_est']

Layer(name='morph_analysis_est', attributes=('normaliseeritud_sõne', 'algvorm', 'lõpp', 'sõnaliik', 'vormi_nimetus', 'kliitik'), spans=SL[Span('Aga', [{'normaliseeritud_sõne': 'Aga', 'algvorm': 'aga', 'lõpp': '0', 'sõnaliik': 'sidesõna', 'vormi_nimetus': '', 'kliitik': ''}]),
Span('kõik', [{'normaliseeritud_sõne': 'kõik', 'algvorm': 'kõik', 'lõpp': '0', 'sõnaliik': 'asesõna', 'vormi_nimetus': 'mitmus nimetav (nominatiiv)', 'kliitik': ''}, {'normaliseeritud_sõne': 'kõik', 'algvorm': 'kõik', 'lõpp': '0', 'sõnaliik': 'asesõna', 'vormi_nimetus': 'ainsus nimetav (nominatiiv)', 'kliitik': ''}]),
Span('juhtus', [{'normaliseeritud_sõne': 'juhtus', 'algvorm': 'juhtuma', 'lõpp': 's', 'sõnaliik': 'tegusõna', 'vormi_nimetus': 'kindel kõneviis lihtminevik 3. isik ainsus aktiiv jaatav kõne', 'kliitik': ''}]),
Span('iseenesest', [{'normaliseeritud_sõne': 'iseenesest', 'algvorm': 'iseenesest', 'lõpp': '0', 'sõnaliik': 'määrsõna', 'vormi_nimetus': '', 'kliitik': ''}]),
Span('.', [{'normaliseeritud_sõne': '.', 'algvorm': '.', 'lõpp': '', 'sõnaliik': 'lausemärk', 'vormi_nimetus': '', 'kliitik': ''}])])

However, note that the layer `'morph_analysis_est'` is provided for educational purposes, and it is not standard in EstNLTK. 
All the tools building upon morphological analysis are using the layer `'morph_analysis'` .
In the following examples, we will also continue with the standard layer.

### Accessing details of the morphological analysis 

`text['morph_analysis']` and `text.morph_analysis` give us the whole layer of morphological analyses. 
If we are only interested in specific details, e.g. in partofspeechtags or lemmas, we can use attributes to access them:

In [16]:
text.partofspeech

,partofspeech
0,J
1,P
,P
2,V
3,D
4,Z


In [17]:
text.lemma

,lemma
0,aga
1,kõik
,kõik
2,juhtuma
3,iseenesest
4,.


Resulting `AmbiguousAttributeList` can also be converted to a list:

In [18]:
list(text.lemma)

[['aga'], ['kõik', 'kõik'], ['juhtuma'], ['iseenesest'], ['.']]

Using indexing on `text.morph_analysis`, we can take out analyses of specific words.
For instance, let's take out the 2nd word and its lemmas and forms:

In [19]:
print('word:',   text.morph_analysis[1].text)
print('lemmas:', text.morph_analysis[1].lemma)
print('forms:',  text.morph_analysis[1].form)

word: kõik
lemmas: ['kõik', 'kõik']
forms: ['pl n', 'sg n']


In [20]:
# or use zip to combine lemmas and forms of the 2nd word
list( zip(text.morph_analysis[1].lemma, text.morph_analysis[1].form) )

[('kõik', 'pl n'), ('kõik', 'sg n')]

Because 'words' is parent of 'morph_analysis' layer, we can also access analyses through words:

In [21]:
# lemmas of all words
text.words.lemma

,lemma
0,aga
1,kõik
,kõik
2,juhtuma
3,iseenesest
4,.


Taking full advantage of the relations between layers, we can iterate over sentences of `Text`, and then access morphological analyses from words of the sentences:

In [22]:
text = Text("Olin nägin vaatasin. Ja väga hea oli.").tag_layer()

In [23]:
for sentence in text.sentences:
    print(' Sentence: ', sentence.enclosing_text)
    for word in sentence:
        # Output first lemma and partofspeech of the word
        print( word.morph_analysis.lemma[0], word.morph_analysis.partofspeech[0] )
    print()

 Sentence:  Olin nägin vaatasin.
olema V
nägema V
vaatama V
. Z

 Sentence:  Ja väga hea oli.
ja J
väga D
hea A
olema V
. Z



### Parameters of morphological analysis: disambiguation, guessing...

By default, EstNLTK performs morphological analysis with disambiguation (giving out the analysis that is correct in the context), guessing (if the word is not in the dictionary and cannot be resolved as a compound, it is given a 'guessed' analysis) and proper name analysis. While this kind of output is easy to use because each word has been given an analysis and most words receive only one analysis,  sometimes we want to use the analyser differently. If we want to enhance the recall of the morphological analysis, we can switch off the disambiguation (which, of course, also affects precision). We can also switch off guessing and proper name analysis e.g to verify whether a word exists in Estonian or not. And finally, if we switch on the text-based disambiguation, we will likely get better disambiguation results (especially on proper names), although this may not work on every corpora (e.g. usually works well on news articles, but be careful when applying this on the Internet language).

To change the parameters of morphological analyser, we can use a resolver. 
Resolver contains information about how to create a layer: what dependency layers are required, and what is the tagger responsible for creating the layer.

You can access the default resolver via `layer_resolver` attribute of `Text`:

In [24]:
Text.layer_resolver

layer,depends_on,tagger_name,description
tokens,[],TokensTagger,Preprocessing for word segmentation: segments text into tokens.
compound_tokens,[tokens],CompoundTokenTagger,Preprocessing for word segmentation: joins tokens into compound tokens.
words,"[tokens, compound_tokens]",WordTagger,Segments text into words.
sentences,"[compound_tokens, words]",SentenceTokenizer,Segments text into sentences.
paragraphs,[sentences],ParagraphTokenizer,Segments text into paragraphs.
morph_analysis,"[compound_tokens, words, sentences]",VabamorfTagger,Tags morphological analysis with Vabamorf.
clauses,"[words, sentences, morph_analysis]",ClauseSegmenter,Segments sentences into clauses. (requires Java)
morph_analysis_est,[morph_analysis],VabamorfEstCatConverter,Translates category names of Vabamorf's morphological analyses into Estonian (for educational purposes).
morph_extended,[morph_analysis],MorphExtendedTagger,Converts Vabamorf's morphological analyses to syntax preprocessing (CG3) format.
gt_morph_analysis,"[words, sentences, morph_analysis, clauses]",GTMorphConverter,Converts Vabamorf's morphological analyses to giellatekno's (GT) format.


Resolver has method `get_tagger`, which returns the tagger responsible for creating the given layer. 
We can use it to check the default parameters of morph_analysis:

In [25]:
Text.layer_resolver.get_tagger('morph_analysis')

VabamorfTagger(input_layers=('words', 'sentences', 'compound_tokens'), output_layer=morph_analysis, output_attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech'), guess=True, propername=True, disambiguate=True, compound=True, phonetic=False, slang_lex=False, postanalysis_tagger=PostMorphAnalysisTagger(('compound_tokens', 'words', 'morph_analysis')->morph_analysis), use_postanalysis=True, analysis_reorderer=MorphAnalysisReorderer(('morph_analysis',)->morph_analysis), use_reorderer=True, textbased_disambiguator=CorpusBasedMorphDisambiguator(['words', 'sentences', 'morph_analysis']*->morph_analysis*), predisambiguate=False, postdisambiguate=False)

Basic configuration parameters are:

* `guess` -- if a word is not in the dictionary and cannot be resolved as a compound, then it's analyses will be guessed;
* `propername` -- titlecase words will receive additional guesses for propername analyses;
* `disambiguate` -- if there are multiple possible analyses for a word, then only analyses fitting to the context will be picked out. This leaves you only one analysis per word for most words;
* `compound` -- roots in analyses will have compound word markers. Normally, you wouldn't need to change this parameter;
* `phonetic` -- roots in analyses will have phonetic markers. Normally, you wouldn't need to change this parameter;

Other parameters are related to components which enhance the quality of morphological analysis:

* Parameter `slang_lex` switches on an extended version of Vabamorf's lexicon, which contains extra entries for analysing most common spoken and slang words, such as _'muideks'_ , _'kodukas'_ , _'mõnsa'_ , _'mersu'_ , _'kippelt'_ .


* Parameter `postanalysis_tagger` refers to an internal component of `VabamorfTagger`, which makes post-corrections to morphological analyses. Details are covered in the tutorial [01_morphological_analysis.ipynb](../nlp_pipeline/B_morphology/01_morphological_analysis.ipynb). The component can be enabled/disabled by the flag `use_postanalysis`. 


* Parameter `analysis_reorderer` refers to an internal component, which re-orders ambiguous analyses by their corpus frequency (based on [Estonian UD corpus](https://github.com/estnltk/estnltk-model-training/tree/main/ud_morph_tools/amb_morph_reordering)). The reordering is applied as a last step, after the disambiguation. Details are covered in the tutorial [03_morph_analysis_reordering.ipynb](../nlp_pipeline/B_morphology/03_morph_analysis_reordering.ipynb). The component can be enabled/disabled by the flag `use_reorderer`. 


* Parameter `textbased_disambiguator` refers to an internal component, which analyses ambiguities in the whole text in order to make advanced disambiguation decisions. It consists of two sub-steps. First, pre-disambiguation of ambiguous proper name analyses applied before the standard disambiguation (flag `predisambiguate`). Second, post-disambiguation of remaining ambiguous analyses applied after the standard disambiguation (flag `postdisambiguate`). Details are in the tutorial [04_morph_analysis_with_corpus-based_disambiguation.ipynb](../nlp_pipeline/B_morphology/04_morph_analysis_with_corpus-based_disambiguation.ipynb).

The configuration tells us that by default, disambiguation, proper name analysis, compound word analysis, and guessing are all applied during morphological analysis.

A special function `make_resolver` can be used to create a new instance of the default resolver. 
This function also accepts the configuration parameters of morpohological analysis, so you can create a resolver that has different morpohological analysis settings:

In [26]:
from estnltk.default_resolver import make_resolver

In [27]:
# create a new resolver with different morph_analysis settings
resolver = make_resolver(
                 disambiguate=True,
                 guess=True,
                 propername=True,
                 phonetic=False,
                 compound=True)

Using the new resolver, we can write an equivalent code to
```python
text.tag_layer()
```
as follows:

In [28]:
text = Text("Kärbes hulbib mees ja naeris puhub sädelevaid mulle.")

In [29]:
text.tag_layer(resolver=resolver)['morph_analysis']

Layer(name='morph_analysis', attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech'), spans=SL[Span('Kärbes', [{'normalized_text': 'Kärbes', 'lemma': 'kärbes', 'root': 'kärbes', 'root_tokens': ['kärbes'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'S'}]),
Span('hulbib', [{'normalized_text': 'hulbib', 'lemma': 'hulpima', 'root': 'hulpi', 'root_tokens': ['hulpi'], 'ending': 'b', 'clitic': '', 'form': 'b', 'partofspeech': 'V'}]),
Span('mees', [{'normalized_text': 'mees', 'lemma': 'mees', 'root': 'mees', 'root_tokens': ['mees'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'S'}]),
Span('ja', [{'normalized_text': 'ja', 'lemma': 'ja', 'root': 'ja', 'root_tokens': ['ja'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'J'}]),
Span('naeris', [{'normalized_text': 'naeris', 'lemma': 'naerma', 'root': 'naer', 'root_tokens': ['naer'], 'ending': 'is', 'clitic': '', 'form': 's', 'partofspeech': 'V'}]),
Span('puhub', [{'normalized_text': 'puhub', 'lemma': 'puhuma', 'root': 'puhu', 'root_tokens': ['puhu'], 'ending': 'b', 'clitic': '', 'form': 'b', 'partofspeech': 'V'}]),
Span('sädelevaid', [{'normalized_text': 'sädelevaid', 'lemma': 'sädelev', 'root': 'sädelev', 'root_tokens': ['sädelev'], 'ending': 'id', 'clitic': '', 'form': 'pl p', 'partofspeech': 'A'}]),
Span('mulle', [{'normalized_text': 'mulle', 'lemma': 'mina', 'root': 'mina', 'root_tokens': ['mina'], 'ending': 'lle', 'clitic': '', 'form': 'sg all', 'partofspeech': 'P'}]),
Span('.', [{'normalized_text': '.', 'lemma': '.', 'root': '.', 'root_tokens': ['.'], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}])])

As we can see from the result, with default morphological analysis, all the words get assigned exactly one analysis, but three of them are not correct. The disambiguator has wrongly deleted the correct analyses this time. 

*Note that this example sentence is a little out of the ordinary and hence the bad performance of disambiguator. The more 'normal' your text is, the better the results.*

If we want to change the parameters of morphological analysis, we have to change the default values of the flags to create a customized resolver. For example, to switch off disambiguation, we have to set this value to False:

In [30]:
resolver2 = make_resolver(
                 disambiguate=False,
                 guess=True,
                 propername=True,
                 phonetic=False,
                 compound=True)

The resolver tags only those layers on the text that have not been previously tagged. To see the effect of changed parameters we have to create a new text object or delete the affected layer.

In [31]:
text.pop_layer('morph_analysis')  # remove morph_analysis from text 
text.tag_layer(resolver=resolver2)['morph_analysis']

Layer(name='morph_analysis', attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech', '_ignore'), spans=SL[Span('Kärbes', [{'normalized_text': 'Kärbes', 'lemma': 'Kärbe', 'root': 'Kärbe', 'root_tokens': ['Kärbe'], 'ending': 's', 'clitic': '', 'form': 'sg in', 'partofspeech': 'H', '_ignore': False}, {'normalized_text': 'Kärbes', 'lemma': 'Kärbes', 'root': 'Kärbes', 'root_tokens': ['Kärbes'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'H', '_ignore': False}, {'normalized_text': 'Kärbes', 'lemma': 'kärbes', 'root': 'kärbes', 'root_tokens': ['kärbes'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'S', '_ignore': False}]),
Span('hulbib', [{'normalized_text': 'hulbib', 'lemma': 'hulpima', 'root': 'hulpi', 'root_tokens': ['hulpi'], 'ending': 'b', 'clitic': '', 'form': 'b', 'partofspeech': 'V', '_ignore': False}]),
Span('mees', [{'normalized_text': 'mees', 'lemma': 'mees', 'root': 'mees', 'root_tokens': ['mees'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'S', '_ignore': False}, {'normalized_text': 'mees', 'lemma': 'mesi', 'root': 'mesi', 'root_tokens': ['mesi'], 'ending': 's', 'clitic': '', 'form': 'sg in', 'partofspeech': 'S', '_ignore': False}]),
Span('ja', [{'normalized_text': 'ja', 'lemma': 'ja', 'root': 'ja', 'root_tokens': ['ja'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'J', '_ignore': False}]),
Span('naeris', [{'normalized_text': 'naeris', 'lemma': 'naerma', 'root': 'naer', 'root_tokens': ['naer'], 'ending': 'is', 'clitic': '', 'form': 's', 'partofspeech': 'V', '_ignore': False}, {'normalized_text': 'naeris', 'lemma': 'naeris', 'root': 'naeris', 'root_tokens': ['naeris'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'S', '_ignore': False}, {'normalized_text': 'naeris', 'lemma': 'naeris', 'root': 'naeris', 'root_tokens': ['naeris'], 'ending': 's', 'clitic': '', 'form': 'sg in', 'partofspeech': 'S', '_ignore': False}]),
Span('puhub', [{'normalized_text': 'puhub', 'lemma': 'puhuma', 'root': 'puhu', 'root_tokens': ['puhu'], 'ending': 'b', 'clitic': '', 'form': 'b', 'partofspeech': 'V', '_ignore': False}]),
Span('sädelevaid', [{'normalized_text': 'sädelevaid', 'lemma': 'sädelev', 'root': 'sädelev', 'root_tokens': ['sädelev'], 'ending': 'id', 'clitic': '', 'form': 'pl p', 'partofspeech': 'A', '_ignore': False}]),
Span('mulle', [{'normalized_text': 'mulle', 'lemma': 'mull', 'root': 'mull', 'root_tokens': ['mull'], 'ending': 'e', 'clitic': '', 'form': 'pl p', 'partofspeech': 'S', '_ignore': False}, {'normalized_text': 'mulle', 'lemma': 'mina', 'root': 'mina', 'root_tokens': ['mina'], 'ending': 'lle', 'clitic': '', 'form': 'sg all', 'partofspeech': 'P', '_ignore': False}, {'normalized_text': 'mulle', 'lemma': 'mulle', 'root': 'mulle', 'root_tokens': ['mulle'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'S', '_ignore': False}]),
Span('.', [{'normalized_text': '.', 'lemma': '.', 'root': '.', 'root_tokens': ['.'], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z', '_ignore': False}])])

##### Note:
 * If disambiguation is switched off, the layer `morph_analysis` will have one extra attribute named `_ignore`. This is actually an internal attribute that is used to tell the disambiguator which analyses should be ignored. Once the disambiguation will be applied, the attribute will be removed. ( You can use `VabamorfDisambiguator` to perform disambiguation separately, see the details in the tutorial [01_morphological_analysis.ipynb](../nlp_pipeline/B_morphology/01_morphological_analysis.ipynb) );

##### Warning:
 * Changing parameters of morphological analysis also affects the performance of downstream taggers that are depending on the results of morphological analysis (such as named entity recognition and syntactic parsing). To the extent that some taggers become not applicable on the resulting morph_analysis layer. Therefore, if you change parameters of morphological analysis, it is not recommended to use any taggers depending on morph_analysis (or use these taggers only when you know what you are doing);

#### Unknown words

If guessings and disambiguation are switched off ( `guess=False`, `propername=False` and `disambiguate=False` ), then morphological analyser can be used to detect unknown words -- words that are orthographically incorrect, or not common in written language:

In [32]:
from estnltk.default_resolver import make_resolver

# Switch off guessing and disambiguation
resolver3 = make_resolver(
                 disambiguate=False,
                 guess=False,
                 propername=False,
                 phonetic=False,
                 compound=True)

# Tag morph analysis
text = Text("Ma tahax minna järve ääde")
text.tag_layer(resolver=resolver3)['morph_analysis']

Layer(name='morph_analysis', attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech', '_ignore'), spans=SL[Span('Ma', [{'normalized_text': 'Ma', 'lemma': 'mina', 'root': 'mina', 'root_tokens': ['mina'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'P', '_ignore': False}]),
Span('tahax', [{'normalized_text': None, 'lemma': None, 'root': None, 'root_tokens': None, 'ending': None, 'clitic': None, 'form': None, 'partofspeech': None, '_ignore': False}]),
Span('minna', [{'normalized_text': 'minna', 'lemma': 'minema', 'root': 'mine', 'root_tokens': ['mine'], 'ending': 'a', 'clitic': '', 'form': 'da', 'partofspeech': 'V', '_ignore': False}]),
Span('järve', [{'normalized_text': 'järve', 'lemma': 'järv', 'root': 'järv', 'root_tokens': ['järv'], 'ending': '0', 'clitic': '', 'form': 'adt', 'partofspeech': 'S', '_ignore': False}, {'normalized_text': 'järve', 'lemma': 'järv', 'root': 'järv', 'root_tokens': ['järv'], 'ending': '0', 'clitic': '', 'form': 'sg g', 'partofspeech': 'S', '_ignore': False}, {'normalized_text': 'järve', 'lemma': 'järv', 'root': 'järv', 'root_tokens': ['järv'], 'ending': '0', 'clitic': '', 'form': 'sg p', 'partofspeech': 'S', '_ignore': False}]),
Span('ääde', [{'normalized_text': None, 'lemma': None, 'root': None, 'root_tokens': None, 'ending': None, 'clitic': None, 'form': None, 'partofspeech': None, '_ignore': False}])])

In the previous example, the morphological analysis revealed two unknown words: _'tahax'_ and _'ääde'_. Each unknown still has one analysis, but all attributes of the analysis (lemma, root, ending, partofspeech etc.) are set to `None`.

##### Remarks on morphological analysis:
 * Switching off guessing ( `guess=False` ) only works if guessing of proper names and disambiguation are also switched off  ( `propername=False` and `disambiguate=False` ). If only `guess=False` is used, then the setting is ignored, and the morphological analysis is performed with the default settings;
 * Be aware that: 
    * if `guess` is switched off, then punctuation symbols (such as `'.'`, `'!'`, `'?'`) also do not receive any analyses;
    * **(!)** if `guess` and `propername` are switched off, but disambiguation is switched on, then an exception will be raised if there are unknown words and/or punctuation symbols in the text. This is because disambiguation requires that all words have been morphologically analysed; 
    
      Note also that if you catch the exception, and proceed with the processing, then the Text object will still have layer `morph_analysis`. But the layer will be incomplete, as its analyses will be ambiguous and contain gaps in places of unknown words;
    * **(!)** if `guess` and `propername` are switched off,  and you switch on the parameter `slang_lex` , then slang words, such as `"kudas"` or `"muideks"` , still get analyses and appear as "known words". So, if you want to detect all non-standard words, you should switch off the parameters `guess` and `propername` and refrain from switching on `slang_lex`;
 * Reordering ( `use_reorderer=True` ) only works with `disambiguate=True`;
 * In practice, parameters `compound` and `phonetic` rarely need to be changed. So, it is advisable to change these parameters only when you really know, what you are doing ...

## <span style="color:purple"> Examples</span>

Here, two simple examples of using EstNLTK basic toolchain for extracting relevant parts of text are presented. Let's use the following short text as our corpus:

"Nagu nimigi reedab, on nurgasaag kõige tõhusam tööriist erinevate puitdetailide lõikamiseks, kus eesmärgiks on saavutada täpne lõikenurk ning oluline on lõikenurga seadistamise võimalus. Näiteks pildiraamide meisterdamisel, kus on oluline, et detailide lõikenurgad oleksid kõik täpselt 45 kraadi. Sellisel juhul on nurgasaag täiuslikuks tööriistaks, sest tagab täpsuse ja lõike korratavuse. Üldiselt on valdav osa nurgasaage seadistatavad 45-kraadise lõikenurga alla vähemalt ühes suunas. Lisaks võimaldavad mõned saed veel ka saetera kaldenurga seadistamist, mis tuleb kasuks keerukamate detailide lõikamisel. Nurgasaag on väga tõhus ka kitsamate, kuni 30 cm laiuste puulaudade või muude puitdetailide ristlõigete tegemiseks ehk järkamiseks, mida tuleb palju ette näiteks puitkonstruktsioonide ehitamisel või ka näiteks terrassilaudade või puitparketi paigaldamisel."

### Example 1: Finding all different nouns from the text

Let's assume we want to find all different noun lemmas that appear in the text. So, first we have to turn our text into an EstNLTK Text object:

In [33]:
my_text = Text("Nagu nimigi reedab, on nurgasaag kõige tõhusam tööriist erinevate puitdetailide lõikamiseks, kus eesmärgiks on saavutada täpne lõikenurk ning oluline on lõikenurga seadistamise võimalus. Näiteks pildiraamide meisterdamisel, kus on oluline, et detailide lõikenurgad oleksid kõik täpselt 45 kraadi. Sellisel juhul on nurgasaag täiuslikuks tööriistaks, sest tagab täpsuse ja lõike korratavuse. Üldiselt on valdav osa nurgasaage seadistatavad 45-kraadise lõikenurga alla vähemalt ühes suunas. Lisaks võimaldavad mõned saed veel ka saetera kaldenurga seadistamist, mis tuleb kasuks keerukamate detailide lõikamisel. Nurgasaag on väga tõhus ka kitsamate, kuni 30 cm laiuste puulaudade või muude puitdetailide ristlõigete tegemiseks ehk järkamiseks, mida tuleb palju ette näiteks puitkonstruktsioonide ehitamisel või ka näiteks terrassilaudade või puitparketi paigaldamisel.")

In [34]:
my_text

text
"Nagu nimigi reedab, on nurgasaag kõige tõhusam tööriist erinevate puitdetailide lõikamiseks, kus eesmärgiks on saavutada täpne lõikenurk ning oluline on lõikenurga seadistamise võimalus. Näiteks pildiraamide meisterdamisel, kus on oluline, et detailide lõikenurgad oleksid kõik täpselt 45 kraadi. Sellisel juhul on nurgasaag täiuslikuks tööriistaks, sest tagab täpsuse ja lõike korratavuse. Üldiselt on valdav osa nurgasaage seadistatavad 45-kraadise lõikenurga alla vähemalt ühes suunas. Lisaks võimaldavad mõned saed veel ka saetera kaldenurga seadistamist, mis tuleb kasuks keerukamate detailide lõikamisel. Nurgasaag on väga tõhus ka kitsamate, kuni 30 cm laiuste puulaudade või muude puitdetailide ristlõigete tegemiseks ehk järkamiseks, mida tuleb palju ette näiteks puitkonstruktsioonide ehitamisel või ka näiteks terrassilaudade või puitparketi paigaldamisel."


Next, we need to let the taggers do their job. Let's use the automatic tag_layer() method for that:

In [35]:
my_text.tag_layer()

Text(text='Nagu nimigi reedab, on nurgasaag kõige tõhusam tööriist erinevate puitdetailide lõikamiseks, kus eesmärgiks on saavutada täpne lõikenurk ning oluline on lõikenurga seadistamise võimalus. Näiteks pildiraamide meisterdamisel, kus on oluline, et detailide lõikenurgad oleksid kõik täpselt 45 kraadi. Sellisel juhul on nurgasaag täiuslikuks tööriistaks, sest tagab täpsuse ja lõike korratavuse. Üldiselt on valdav osa nurgasaage seadistatavad 45-kraadise lõikenurga alla vähemalt ühes suunas. Lisaks võimaldavad mõned saed veel ka saetera kaldenurga seadistamist, mis tuleb kasuks keerukamate detailide lõikamisel. Nurgasaag on väga tõhus ka kitsamate, kuni 30 cm laiuste puulaudade või muude puitdetailide ristlõigete tegemiseks ehk järkamiseks, mida tuleb palju ette näiteks puitkonstruktsioonide ehitamisel või ka näiteks terrassilaudade või puitparketi paigaldamisel.')

Now we can iterate over the lemmas and part-of-speech tags to extract the lemmas that are tagged as nouns. For this, it's easiest to use the zip function:

In [36]:
noun_lemmas = []
for lemma, postag in zip(my_text.lemma, my_text.partofspeech):
    if 'S' in postag:
        noun_lemmas += lemma

**NB!** Note that both, lemmas and part-of-speech tags for every word are given as a list, that's why we have to check if 'S' is in the postag and we cannot use `append` for adding the lemma to noun_lemmas list without iterating over each word's part-of-speech tags and lemmas.

In [37]:
noun_lemmas

['nimi',
 'nurgasaag',
 'tööriist',
 'puitdetail',
 'lõikamine',
 'eesmärk',
 'lõikenurk',
 'lõikenurk',
 'seadistamine',
 'võimalus',
 'näide',
 'pildiraam',
 'meisterdamine',
 'detail',
 'lõikenurk',
 'kraad',
 'juht',
 'nurgasaag',
 'tööriist',
 'täpsus',
 'lõige',
 'korratavus',
 'osa',
 'nurgasaag',
 'lõikenurk',
 'suund',
 'lisa',
 'saag',
 'saetera',
 'kaldenurk',
 'seadistamine',
 'kasu',
 'detail',
 'lõikamine',
 'nurgasaag',
 'laius',
 'puulaud',
 'puitdetail',
 'ristlõige',
 'tegemine',
 'järkamine',
 'näide',
 'puitkonstruktsioon',
 'ehitamine',
 'näide',
 'terrassilaud',
 'puitparkett',
 'paigaldamine']

In this case, we can easily guess what the specific text was about. If we had a larger corpus, we could make a frequency list and e.g draw some conclusions about topics/word use in a specific publication or variety of language in general.

### Example 2: Finding all sentences that contain an infinitive verb

Let's assume we want to extract all sentences containing an infinitive verb form from a corpus. Let's use the same text as in the previous example. Therefore, we have already tagged the necessary layers and we can iterate over the forms and extract sentences that we want to study:

In [38]:
infinitive_sentences = []
for sent in my_text.sentences:
    for form in sent.form:
        if 'da' in form:
            a = sent.enclosing_text
            infinitive_sentences.append(a)
            break # not to include the same sentence twice)
infinitive_sentences

['Nagu nimigi reedab, on nurgasaag kõige tõhusam tööriist erinevate puitdetailide lõikamiseks, kus eesmärgiks on saavutada täpne lõikenurk ning oluline on lõikenurga seadistamise võimalus.']

##  <span style="color:purple"> Further details </span>

Further details about EstNLTK's NLP tools can be found from [nlp pipeline tutorials](../nlp_pipeline).